<a href="https://colab.research.google.com/github/sreehari59/NxtGen_AcciVision/blob/main/DPS_Task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
os.listdir()

['.config',
 '.ipynb_checkpoints',
 'monatszahlen2405_verkehrsunfaelle_export_31_05_24_r.csv',
 'sample_data']

In [ ]:
import pandas as pd
df = pd.read_csv("monatszahlen2405_verkehrsunfaelle_export_31_05_24_r.csv")
df

,MONATSZAHL,AUSPRAEGUNG,JAHR,MONAT,WERT,VORJAHRESWERT,VERAEND_VORMONAT_PROZENT,VERAEND_VORJAHRESMONAT_PROZENT,ZWOELF_MONATE_MITTELWERT
0,Alkoholunfälle,insgesamt,2024,202401,NaN,NaN,NaN,NaN,NaN
1,Alkoholunfälle,insgesamt,2024,202402,NaN,NaN,NaN,NaN,NaN
2,Alkoholunfälle,insgesamt,2024,202403,NaN,NaN,NaN,NaN,NaN
3,Alkoholunfälle,insgesamt,2024,202404,NaN,NaN,NaN,NaN,NaN
4,Alkoholunfälle,insgesamt,2024,202405,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
2249,Verkehrsunfälle,Verletzte und Getötete,2000,200008,647.0,NaN,-13.04,NaN,584.0
2250,Verkehrsunfälle,Verletzte und Getötete,2000,200009,675.0,NaN,4.33,NaN,594.0
2251,Verkehrsunfälle,Verletzte und Getötete,2000,200010,615.0,NaN,-8.89,NaN,596.0
2252,Verkehrsunfälle,Verletzte und Getötete,2000,200011,578.0,NaN,-6.02,NaN,594.0


In [ ]:
df.columns

Index(['MONATSZAHL', 'AUSPRAEGUNG', 'JAHR', 'MONAT', 'WERT', 'VORJAHRESWERT',
       'VERAEND_VORMONAT_PROZENT', 'VERAEND_VORJAHRESMONAT_PROZENT',
       'ZWOELF_MONATE_MITTELWERT'],
      dtype='object')


1. MONATSZAHL - MONTH NUMBER
2. AUSPRAEGUNG - PRONUNCIATION
3. JAHR - YEAR
4. MONAT - MONTH
5. WERT - VALUE
6. VORJAHRESWERT - PREVIOUS YEAR VALUE
7. VERAEND_VORMONAT_PROZENT - CHANGE PREVIOUS MONTH PERCENTAGE
8. VERAEND_VORJAHRESMONAT_PROZENT -CHANGE FROM PREVIOUS YEAR MONTH PERCENTAGE
9. ZWOELF_MONATE_MITTELWERT - TWELVE MONTHS AVERAGE







In [ ]:
df.drop(['VORJAHRESWERT',
       'VERAEND_VORMONAT_PROZENT', 'VERAEND_VORJAHRESMONAT_PROZENT',
       'ZWOELF_MONATE_MITTELWERT'],axis=1, inplace=True)

In [ ]:
df["AUSPRAEGUNG"].unique()

array(['insgesamt', 'Verletzte und Getötete', 'mit Personenschäden'],
      dtype=object)

In [ ]:
df["MONATSZAHL"].unique()
#'Alcohol accidents', 'escape accidents', 'traffic accidents'

array(['Alkoholunfälle', 'Fluchtunfälle', 'Verkehrsunfälle'], dtype=object)

In [ ]:
df[df["JAHR"]==2020]

,MONATSZAHL,AUSPRAEGUNG,JAHR,MONAT,WERT
50,Alkoholunfälle,insgesamt,2020,Summe,430.0
51,Alkoholunfälle,insgesamt,2020,202001,28.0
52,Alkoholunfälle,insgesamt,2020,202002,40.0
53,Alkoholunfälle,insgesamt,2020,202003,27.0
54,Alkoholunfälle,insgesamt,2020,202004,26.0
...,...,...,...,...,...
1990,Verkehrsunfälle,Verletzte und Getötete,2020,202008,563.0
1991,Verkehrsunfälle,Verletzte und Getötete,2020,202009,634.0
1992,Verkehrsunfälle,Verletzte und Getötete,2020,202010,566.0
1993,Verkehrsunfälle,Verletzte und Getötete,2020,202011,348.0


In [ ]:
df[(df["JAHR"]==2020) & (df["MONAT"]=="Summe")]

,MONATSZAHL,AUSPRAEGUNG,JAHR,MONAT,WERT
50,Alkoholunfälle,insgesamt,2020,Summe,430.0
372,Alkoholunfälle,Verletzte und Getötete,2020,Summe,250.0
694,Fluchtunfälle,insgesamt,2020,Summe,10024.0
1016,Fluchtunfälle,Verletzte und Getötete,2020,Summe,465.0
1338,Verkehrsunfälle,insgesamt,2020,Summe,38434.0
1660,Verkehrsunfälle,mit Personenschäden,2020,Summe,4918.0
1982,Verkehrsunfälle,Verletzte und Getötete,2020,Summe,5704.0


In [ ]:
df[(df["JAHR"]==2020) & (df["MONATSZAHL"]=="Alkoholunfälle")]

,MONATSZAHL,AUSPRAEGUNG,JAHR,MONAT,WERT
50,Alkoholunfälle,insgesamt,2020,Summe,430.0
51,Alkoholunfälle,insgesamt,2020,202001,28.0
52,Alkoholunfälle,insgesamt,2020,202002,40.0
53,Alkoholunfälle,insgesamt,2020,202003,27.0
54,Alkoholunfälle,insgesamt,2020,202004,26.0
55,Alkoholunfälle,insgesamt,2020,202005,40.0
56,Alkoholunfälle,insgesamt,2020,202006,49.0
57,Alkoholunfälle,insgesamt,2020,202007,58.0
58,Alkoholunfälle,insgesamt,2020,202008,46.0
59,Alkoholunfälle,insgesamt,2020,202009,46.0


In [ ]:
df[(df["JAHR"]==2020) & (df["AUSPRAEGUNG"]=="insgesamt")]

,MONATSZAHL,AUSPRAEGUNG,JAHR,MONAT,WERT
50,Alkoholunfälle,insgesamt,2020,Summe,430.0
51,Alkoholunfälle,insgesamt,2020,202001,28.0
52,Alkoholunfälle,insgesamt,2020,202002,40.0
53,Alkoholunfälle,insgesamt,2020,202003,27.0
54,Alkoholunfälle,insgesamt,2020,202004,26.0
55,Alkoholunfälle,insgesamt,2020,202005,40.0
56,Alkoholunfälle,insgesamt,2020,202006,49.0
57,Alkoholunfälle,insgesamt,2020,202007,58.0
58,Alkoholunfälle,insgesamt,2020,202008,46.0
59,Alkoholunfälle,insgesamt,2020,202009,46.0


In [ ]:
for i in df["JAHR"].unique():
  print("Number of rows for year:",i,"is",df[df["JAHR"]==i].shape[0])

Number of rows for year: 2024 is 84
Number of rows for year: 2023 is 84
Number of rows for year: 2022 is 91
Number of rows for year: 2021 is 91
Number of rows for year: 2020 is 91
Number of rows for year: 2019 is 91
Number of rows for year: 2018 is 91
Number of rows for year: 2017 is 91
Number of rows for year: 2016 is 91
Number of rows for year: 2015 is 91
Number of rows for year: 2014 is 91
Number of rows for year: 2013 is 91
Number of rows for year: 2012 is 91
Number of rows for year: 2011 is 91
Number of rows for year: 2010 is 91
Number of rows for year: 2009 is 91
Number of rows for year: 2008 is 91
Number of rows for year: 2007 is 91
Number of rows for year: 2006 is 91
Number of rows for year: 2005 is 91
Number of rows for year: 2004 is 91
Number of rows for year: 2003 is 91
Number of rows for year: 2002 is 91
Number of rows for year: 2001 is 91
Number of rows for year: 2000 is 84


In [ ]:
# Plotting the number of accidents per category
import plotly.express as px

fig = px.line(df[(df["AUSPRAEGUNG"]=="insgesamt") & (df["MONAT"]=="Summe")],
              x="JAHR", y="WERT", color='MONATSZAHL', markers=True)
fig.show()

In [ ]:
fig = px.bar(df[(df["AUSPRAEGUNG"]=="insgesamt") & (df["MONAT"]=="Summe")],
              x="JAHR", y="WERT", color='MONATSZAHL', title="Long-Form Input")
fig.show()

## Model to Forecast

In [ ]:
new_df = df[df["JAHR"] < 2021]
new_df.head(2)

,MONATSZAHL,AUSPRAEGUNG,JAHR,MONAT,WERT
50,Alkoholunfälle,insgesamt,2020,Summe,430.0
51,Alkoholunfälle,insgesamt,2020,202001,28.0


In [ ]:
print("Size of required data: ",new_df.shape)

Size of required data:  (1904, 5)


In [ ]:
# Remove null values
new_df = new_df.dropna()
print("Size of required data after dropping null values: ",new_df.shape)

Size of required data after dropping null values:  (1904, 5)


In [ ]:
# Remove 'Summe' value from the MONAT column
new_df = new_df[new_df["MONAT"]!="Summe"]
new_df.head(2)

,MONATSZAHL,AUSPRAEGUNG,JAHR,MONAT,WERT
51,Alkoholunfälle,insgesamt,2020,202001,28.0
52,Alkoholunfälle,insgesamt,2020,202002,40.0


In [ ]:
from pycaret.classification import *